In [1]:
import theano
import theano.tensor as T
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Using cuDNN version 5105 on context None
Mapped name None to device cuda: GeForce GT 740M (0000:01:00.0)


In [3]:
from utils import get_network_str,load_weights
import lasagne
from symbols import make_net
import theano.tensor as T


def make_feature_fn(model_path='models/gmm099_loss-234.46',ndim=24):
    data = T.tensor4(name='data')
    net = make_net(data,ndim,use_score=True)
    load_weights(net,model_path)
    net_output = lasagne.layers.get_output(net)
    net_output = T.transpose(net_output,(0,2,3,1))
    X = net_output[:,:,:,1:]
    general_dist = net_output[:,:,:,:1]
    X = X/(T.sqrt(T.square(X).sum(3)+np.float32(1e-8))[:,:,:,None])
    net_output = T.concatenate([general_dist,X],axis=3)    
    return  theano.function([data],net_output)

features_fn = make_feature_fn()

In [4]:
from gmm_bg_substructor import BackgroundSubtraction

classical_params =  { 'algorithm': 'grimson_gmm', 
                        'low': 3.0 * 3.0,
                        'high': 3.0 * 3.0 * 2,
                        'alpha': 0.01,
                        'max_modes': 3,
                        'channels': 2,
                        'variance': 36.,
                        'bg_threshold': 0.75,
                        'min_variance': 4.,
                        'variance_factor': 5.}

deep_params =  { 'algorithm': 'grimson_gmm', 
                        'low': 20.,#*24*24,
                        'high': 30,#.*24*24,
                        'alpha': 0.01,
                        'max_modes': 3,
                        'channels': 2,
                        'variance': 3.,
                        'bg_threshold': 0.75,
                        'min_variance': .5,
                        'variance_factor': 3.}


class ClassicBG:
    def __init__(self,params):
        self.bg_sub = BackgroundSubtraction()
        self.init = False
        self.params = params
        self.i = 0 

    def update(self,img):
        img = img.astype(np.float32)
        high_threshold_mask = np.zeros(shape=img.shape[0:2], dtype=np.uint8)
        low_threshold_mask = np.zeros_like(high_threshold_mask)
        self.params['channels'] = img.shape[-1]
        if(not  self.init):
            self.bg_sub.init_model(img, self.params)
            self.init = True
        self.bg_sub.subtract(self.i, img, low_threshold_mask, high_threshold_mask)
        self.i = self.i+1
        return low_threshold_mask

In [9]:

bg_sub = ClassicBG(classical_params)


def make_test(out_dir,dataset='dataset',max_frames=500,im_size = (320,240)):
    for d in os.listdir(dataset):
        if not(os.path.isdir(dataset+'/'+d)):
            continue
        folder = d+'/'+os.listdir(dataset+'/'+d)[0]
        out_folder = out_dir+'/'+folder
        make_path(out_folder)
        print folder+' ',
        for i,(img,mask) in enumerate(iterate_video(dataset+'/'+folder)):
            img,mask = resize(img,mask,im_size)
            #data = np.transpose(img[np.newaxis,...],(0,3,1,2)).astype(np.float32)
            #data = features_fn(data)[0]
            #out = bg_sub.update(data)
            out = bg_sub.update(img)
            cv2.imwrite(out_folder+'/'+str(i)+'.jpg',out)
            cv2.imwrite(out_folder+'/'+str(i)+'_true.jpg',mask)
            print str(i)+'\r',
            if(i == max_frames):
                break
        print ''
make_test('gmm')

500dWeather/blizzard  0
500eline/highway  0
325eraJitter/badminton  0
500amicBackground/boats  0
intermittentObjectMotion/abandonedBox  
lowFramerate/port_0_17fps  
nightVideos/bridgeEntry  
34Z/continuousPan  0
500dow/backdoor  0
500rmal/corridor  0
turbulence/turbulence0  


In [11]:

bg_sub = ClassicBG(deep_params)
def make_test(out_dir,dataset='dataset',max_frames=500,im_size = (320,240)):
    for d in os.listdir(dataset):
        if not(os.path.isdir(dataset+'/'+d)):
            continue
        folder = d+'/'+os.listdir(dataset+'/'+d)[0]
        out_folder = out_dir+'/'+folder
        make_path(out_folder)
        for i,(img,mask) in enumerate(iterate_video(dataset+'/'+folder)):
            img,mask = resize(img,mask,im_size)
            data = np.transpose(img[np.newaxis,...],(0,3,1,2)).astype(np.float32)
            data = features_fn(data)[0]
            out = bg_sub.update(data)
            cv2.imwrite(out_folder+'/'+str(i)+'.jpg',out)
            cv2.imwrite(out_folder+'/'+str(i)+'_true.jpg',mask)
            print folder+' '+str(i)+'\r',
            if(i == max_frames):
                break
        print ''
make_test('deep_gmm')




PTZ/continuousPan 34
shadow/backdoor 500
thermal/corridor 500



In [79]:
def calc_metrics_img(predict,label):
    mask = (label>230)|(label < 50)
    def prepare(x):
        res = np.zeros_like(x,dtype=np.float32)
        res[x > 240]  = 1
        return res[mask].flatten()
    p = prepare(predict)
    y = prepare(label)
    TP = (p*y).sum()
    TN = ((1-p)*(1-y)).sum()
    FP = (p*(1.0-y)).sum()
    FN = ((1.0-p)*y).sum()
    return np.array([TP,TN,FP,FN])

    
def list_all_jpg(d):
    res = []
    for i in os.listdir(d):
        p = d+'/'+i
        if(os.path.isdir(p)):
            res = res+list_all_jpg(p)
        else:
            if(p[-3:] == 'jpg'):
                if(p.find('true') < 0):
                    res.append(p)
    return res
    
    

def calc_metrics_folder(data_dir):
    res = []
    for j in os.listdir(data_dir):
        if(j.find('true')>=0):
            continue
        pred = cv2.imread(data_dir+'/'+j)
        if(pred.mean() > 240):
            continue
        true = cv2.imread(data_dir+'/'+j[:-4]+'_true.jpg')
        res.append(calc_metrics_img(pred,true))
    return np.array(res).sum(0)

def calc_metric_all_folders(data_dir):
    res = []
    for j in os.listdir(data_dir):
        folder = data_dir+'/'+j+'/'+os.listdir(data_dir+'/'+j)[0]
        res.append(calc_metrics_folder(folder))
        print folder,res[-1]
    return np.array(res).sum(0)
    


def print_total_metric(data_dir):
    res = calc_metric_all_folders(data_dir)
    print res
    TP,TN,FP,FN = res[0],res[1],res[2],res[3]
    results = dict( Recall = TP / (TP + FN),\
            Sp = TN / (TN + FP),\
            FPR = FP / (FP + TN),\
            FNR = FN / (TP + FN),\
            PWC =  100 * (FN + FP) / (TP + FN + FP + TN),\
            F_Measure  =  (2 * (TP / (TP + FP)) * (TP / (TP + FN))) / (TP / (TP + FP) +  TP / (TP + FN)),\
            Precision  = TP / (TP + FP))
    print results

In [71]:
from sklearn import mixture
from gmm_op import calc_log_prob_gmm,calc_log_prob_gmm_componetwise
from numba import jit

def make_predict_fn(ndim=24,param_value = 11.5):
    means,covars,weights = T.matrix(),T.matrix(),T.vector()
    net_output = T.matrix()
    general_dist = net_output[:,:1]
    X = net_output[:,1:]
    p = calc_log_prob_gmm_componetwise(X,means,covars,weights)
    p = T.nnet.softmax(T.concatenate([p,general_dist],axis=1))
    rate = weights/(T.mean(covars,1)+np.float32(1e-8))
    rate_param = theano.shared(np.float32(param_value),name='rate_param',allow_downcast=True)
    soft_sign = T.concatenate([T.tanh(rate- rate_param),-T.ones((1,))])
    
    prediction = T.max((1.+T.sgn(p-T.max(p,1)[:,None]+np.float32(1e-10)))*T.sgn(soft_sign)[None,:],1)/2.
    return theano.function([net_output,means,covars,weights],prediction,allow_input_downcast=True),rate_param

def predict_pixel(pixel,predict_fn,gmm = mixture.GaussianMixture(covariance_type='diag',
                                           n_components=4,
                                           max_iter=2000,
                                           warm_start=False)):
    gmm.fit(pixel[:,1:])
    return predict_fn(pixel,gmm.means_.astype(np.float32),gmm.covariances_.astype(np.float32),gmm.weights_.astype(np.float32))
    

def predict_pixelwise(features,gmms,predict_fn):
    res = np.zeros_like(features[:,:,:,0])
    for i in range(features.shape[1]):
        for j in range(features.shape[2]):
            res[:,i,j] = predict_pixel(features[:,i,j,:],predict_fn,gmms[i*features.shape[2]+j])
    return res


def predict_pixel(pixel,predict_fn,gmm = mixture.GaussianMixture(covariance_type='diag',
                                           n_components=4,
                                           max_iter=2000,
                                           warm_start=False)):
    gmm.fit(pixel[:,1:])
    return predict_fn(pixel,gmm.means_.astype(np.float32),gmm.covariances_.astype(np.float32),gmm.weights_.astype(np.float32))
    

def fit_gmms(features,gmms):
    for i in range(features.shape[1]):
        for j in range(features.shape[2]):
            gmms[i*features.shape[2]+j].fit(features[:,i,j,1:])

@jit
def predict_pixelwise_no_fit(features,gmms,predict_fn):
    res = np.zeros_like(features[:,:,:,0])
    for i in range(features.shape[1]):
        for j in range(features.shape[2]):
            gmm = gmms[i*features.shape[2]+j]
            res[:,i,j] = predict_fn(features[:,i,j,:],gmm.means_,gmm.covariances_,gmm.weights_)
    return res


            
def make_gmms(shape):
    gmms = []
    for i in range(shape[0]):
        for j in range(shape[1]):
            gmms.append(mixture.GaussianMixture(covariance_type='diag',
                               n_components=4,
                               max_iter=1000,
                               warm_start=True))
    return gmms
    
class deepBG:
    def __init__(self,model_path='models/gmm099_loss-234.46',ndim=24,buf_size=200,param_value = 11.5):
        self.gmms = None
        self.predict_fn,self.param = make_predict_fn(ndim,param_value)
        self.feature_fn = make_feature_fn(model_path,ndim)
        self.buffer = None
        self.cur = None
        
    def update(self,img):
        if(self.buffer is None):
            self.buffer = np.empty((self.buf_size,img.shape[0],img.shape[1],self.ndim+1),dtype=np.float32)
        self.buffer[self.cur] = self.features_fn(np.transpose(img[np.newaxis],(0,3,1,2)))
        self.cur = self.cur+1
        if(self.cur == len(self.buffer)):
            res = np.zeros_like(self.buffer[:,:,:,0],dtype=np.uint8)
            self.cur = 0
            if(self.gmms is None):
                self.gmms = make_gmms(self.buffer.shape[1:-1])
            for i in range(img.shape[0]):
                for j in range(img.shape[1]):
                    res[:,i,j] = predict_pixelwise(self.buffer,self.gmms,self.predict_fn).astype(np.uint8)
            return res
        return None

In [46]:
predict_fn,param = make_predict_fn()

In [61]:
folder='/media/data/Programming/motion_segmentation/gmm_segmentation/dataset/badWeather/blizzard'
features = np.empty((256,240//2,320//2,25))
masks = np.empty((256,240//2,320//2),dtype=np.uint8)
for i,(img,mask) in enumerate(iterate_video(folder)):
    if(i == len(features)):
        break
    img,mask = resize(img,mask,(320//2,240//2))
    masks[i] = mask
    features[i] = features_fn(np.transpose(img,(2,0,1))[np.newaxis])[0]
    print str(i)+'\r',

In [63]:
gmms = make_gmms(features.shape[1:-1])
fit_gmms(features,gmms)

In [75]:
from sklearn.metrics import f1_score,accuracy_score
masks[masks < 240] = 0
masks[masks >= 240] = 255
values = [0.1,1,5,10,15,20,30,40,60,80]
res = []
for v in values:
    param.set_value(v)
    out = predict_pixelwise_no_fit(features,gmms,predict_fn)
    out = (1.-out).astype(np.uint8)
    res.append((f1_score((masks/255).flatten(),out.flatten()),accuracy_score((masks/255).flatten(),out.flatten())))
print res

[(0.12582030504728339, 0.95059387207031254), (0.12584593232541399, 0.9505940755208333), (0.13561877667140826, 0.95054850260416668), (0.1944501946867761, 0.95012268066406247), (0.24572559334918007, 0.94912760416666664), (0.27474641778197834, 0.94754394531249997), (0.29643403518660166, 0.94311238606770831), (0.30170315178841323, 0.93843119303385414), (0.29835839881489118, 0.92907755533854164), (0.29591882508096856, 0.92016703287760415)]


In [77]:
np.set_printoptions(4)
print values
print np.array([i[0] for i in res])
print np.array([i[1] for i in res])

[0.1, 1, 5, 10, 15, 20, 30, 40, 60, 80]
[ 0.1258  0.1258  0.1356  0.1945  0.2457  0.2747  0.2964  0.3017  0.2984
  0.2959]
[ 0.9506  0.9506  0.9505  0.9501  0.9491  0.9475  0.9431  0.9384  0.9291
  0.9202]


In [52]:
out_dir = 'out1'
for i in range(len(out)):
    cv2.imwrite(out_dir+'/'+str(i)+'.jpg',((1.-out[i])*255.).astype(np.float32))
    

In [78]:


def make_deep_test(out_dir,dataset='dataset',max_frames=500,im_size = (320//2,240//2)):
    features = np.empty((max_frames,im_size[1],im_size[0],25))
    masks = np.empty((max_frames,im_size[1],im_size[0]),dtype=np.uint8)
    for d in os.listdir(dataset):
        if not(os.path.isdir(dataset+'/'+d)):
            continue
        folder = d+'/'+os.listdir(dataset+'/'+d)[0]
        out_folder = out_dir+'/'+folder
        make_path(out_folder)
        print folder
        for i,(img,mask) in enumerate(iterate_video(dataset+'/'+folder)):
            if(i == len(features)):
                break
            img,mask = resize(img,mask,im_size)
            masks[i] = mask
            features[i] = features_fn(np.transpose(img,(2,0,1))[np.newaxis])[0]
            print str(i)+'\r',
        print 'build gmmms '+str(i)
        gmms = make_gmms(features[:i].shape[1:-1])
        out = ((1.-predict_pixelwise(features[:i],gmms,predict_fn))*255.).astype(np.uint8)
        print folder,'save'
        for j in range(len(out)):
            cv2.imwrite(out_folder+'/'+str(j)+'.jpg',out[j])
            cv2.imwrite(out_folder+'/'+str(j)+'_true.jpg',masks[j])
        print ''

param.set_value(30.)      
make_deep_test('deep_test',max_frames=300)

badWeather/blizzard
build gmmms 300
badWeather/blizzard save

baseline/highway
build gmmms 300
baseline/highway save

cameraJitter/badminton
build gmmms 300
cameraJitter/badminton save

dynamicBackground/boats
build gmmms 300
dynamicBackground/boats save

intermittentObjectMotion/abandonedBox
build gmmms 300
intermittentObjectMotion/abandonedBox save

lowFramerate/port_0_17fps
build gmmms 300
lowFramerate/port_0_17fps save

nightVideos/bridgeEntry
build gmmms 300
nightVideos/bridgeEntry save

PTZ/continuousPan
build gmmms 34
PTZ/continuousPan save

shadow/backdoor
build gmmms 300
shadow/backdoor save

thermal/corridor
build gmmms 300
thermal/corridor save

turbulence/turbulence0
build gmmms 300
turbulence/turbulence0 save



In [82]:
print_total_metric('deep_test')

deep_test/badWeather/blizzard [   189942.  16001814.    733683.    185379.]
deep_test/baseline/highway [   789063.  14771925.    565884.    395760.]
deep_test/cameraJitter/badminton [   248097.  15457650.    880011.    369000.]
deep_test/dynamicBackground/boats [  104499.  7868166.   559671.   251949.]
deep_test/intermittentObjectMotion/abandonedBox [  105726.  7870710.   557127.   250722.]
deep_test/lowFramerate/port_0_17fps [  104547.  7868382.   559455.   251901.]
deep_test/nightVideos/bridgeEntry [  105081.  7872816.   555021.   251367.]
deep_test/PTZ/continuousPan [   55116.  1612356.   114993.   135558.]
deep_test/shadow/backdoor [   507093.  12584601.    742998.    169686.]
deep_test/thermal/corridor [   769965.  13914078.    558678.   1478166.]
deep_test/turbulence/turbulence0 [   770541.  13921050.    551706.   1477590.]
[  3.7497e+06   1.1974e+08   6.3792e+06   5.2171e+06]
{'FNR': 0.58182496, 'FPR': 0.050579499, 'Recall': 0.41817501, 'Sp': 0.94942051, 'Precision': 0.3701953, 

In [80]:
print_total_metric('gmm')

gmm/badWeather/blizzard [  1.3721e+06   1.3023e+08   1.2685e+06   1.1405e+06]
gmm/baseline/highway [  7418463.  95876976.   7768743.    386196.]
gmm/cameraJitter/badminton [  1899516.  59517088.   8178042.    650127.]
gmm/dynamicBackground/boats [  1371741.  46764768.  10378941.   1029987.]
gmm/intermittentObjectMotion/abandonedBox 0.0
gmm/lowFramerate/port_0_17fps 0.0
gmm/nightVideos/bridgeEntry 0.0
gmm/PTZ/continuousPan [  644841.  2094957.  5027727.   130305.]
gmm/shadow/backdoor [  5485191.  84840384.   3148050.    393441.]
gmm/thermal/corridor [  6507534.  95378664.   6923349.   4142880.]
gmm/turbulence/turbulence0 0.0
[  2.4699e+07   5.1470e+08   4.2693e+07   7.8734e+06]
{'FNR': 0.24171783, 'FPR': 0.07659401, 'Recall': 0.75828218, 'Sp': 0.923406, 'Precision': 0.36649919, 'F_Measure': 0.49415790562429901, 'PWC': 8.5710637684844606}


In [81]:
print_total_metric('deep_gmm')

deep_gmm/badWeather/blizzard [  4.4181e+06   2.1229e+08   1.0831e+07   1.1833e+06]
deep_gmm/baseline/highway [  9.9632e+06   1.9953e+08   9.3091e+06   4.5083e+06]
deep_gmm/cameraJitter/badminton [  1845297.  61720504.   9353841.    758718.]
deep_gmm/dynamicBackground/boats [  2.5426e+06   1.0572e+08   9.2348e+06   1.7921e+06]
deep_gmm/intermittentObjectMotion/abandonedBox 0.0
deep_gmm/lowFramerate/port_0_17fps 0.0
deep_gmm/nightVideos/bridgeEntry 0.0
deep_gmm/PTZ/continuousPan [  587382.  2722014.  4202232.   160560.]
deep_gmm/shadow/backdoor [  4216443.  82753968.   5234562.   1662189.]
deep_gmm/thermal/corridor [  6545613.  94683048.   7618995.   4104801.]
deep_gmm/turbulence/turbulence0 0.0
[  3.0119e+07   7.5941e+08   5.5785e+07   1.4170e+07]
{'FNR': 0.31994408, 'FPR': 0.068431124, 'Recall': 0.68005592, 'Sp': 0.93156892, 'Precision': 0.35060957, 'F_Measure': 0.46267990730008174, 'PWC': 8.1391304674815697}
